In [11]:
import os
import pandas as pd
import os
import h5py
from tqdm import tqdm
from omegaconf import OmegaConf

In [ ]:
# ----CONFIG ----
model_path = "data/t15_pretrained_rnn_baseline"
data_dir = "data/hdf5_data_final"
csv_path = "data/t15_copyTaskData_description.csv"
output_file = os.path.join(model_path, "phoneme_text_training_data_final.csv")
# ---------------
# Load metadata
b2txt_csv_df = pd.read_csv(csv_path)
model_args = OmegaConf.load(os.path.join(model_path, "checkpoint/args.yaml"))

def load_h5py_file(file_path, b2txt_csv_df):
    data = {
        'neural_features': [],
        'n_time_steps': [],
        'seq_class_ids': [],
        'seq_len': [],
        'transcriptions': [],
        'sentence_label': [],
        'session': [],
        'block_num': [],
        'trial_num': [],
        'corpus': [],
    }
    # Open the hdf5 file for that day
    with h5py.File(file_path, 'r') as f:

        keys = list(f.keys())

        # For each trial in the selected trials in that day
        for key in keys:
            g = f[key]

            neural_features = g['input_features'][:]
            n_time_steps = g.attrs['n_time_steps']
            seq_class_ids = g['seq_class_ids'][:] if 'seq_class_ids' in g else None
            seq_len = g.attrs['seq_len'] if 'seq_len' in g.attrs else None
            transcription = g['transcription'][:] if 'transcription' in g else None
            sentence_label = g.attrs['sentence_label'][:] if 'sentence_label' in g.attrs else None
            session = g.attrs['session']
            block_num = g.attrs['block_num']
            trial_num = g.attrs['trial_num']

            # match this trial up with the csv to get the corpus name
            year, month, day = session.split('.')[1:]
            date = f'{year}-{month}-{day}'
            row = b2txt_csv_df[(b2txt_csv_df['Date'] == date) & (b2txt_csv_df['Block number'] == block_num)]
            corpus_name = row['Corpus'].values[0]

            data['neural_features'].append(neural_features)
            data['n_time_steps'].append(n_time_steps)
            data['seq_class_ids'].append(seq_class_ids)
            data['seq_len'].append(seq_len)
            data['transcriptions'].append(transcription)
            data['sentence_label'].append(sentence_label)
            data['session'].append(session)
            data['block_num'].append(block_num)
            data['trial_num'].append(trial_num)
            data['corpus'].append(corpus_name)
    return data

LOGIT_TO_PHONEME = [
    'BLANK',
    'AA', 'AE', 'AH', 'AO', 'AW',
    'AY', 'B',  'CH', 'D', 'DH',
    'EH', 'ER', 'EY', 'F', 'G',
    'HH', 'IH', 'IY', 'JH', 'K',
    'L', 'M', 'N', 'NG', 'OW',
    'OY', 'P', 'R', 'S', 'SH',
    'T', 'TH', 'UH', 'UW', 'V',
    'W', 'Y', 'Z', 'ZH',
    ' | ',
]

# Decode true phoneme sequence from seq_class_ids
def decode_phoneme_sequence(seq_ids, seq_len):
    seq = seq_ids[:seq_len]
    seq = [int(p) for p in seq if p != 0]  # remove blanks
    seq = [seq[i] for i in range(len(seq)) if i == 0 or seq[i] != seq[i-1]]  # remove repeats
    return " ".join([LOGIT_TO_PHONEME[p] for p in seq])

# Gather all examples
records = []

for split in ["train", "val"]:  # these contain ground truth phonemes and text
    print(f"\nProcessing split: {split}")

    for session in model_args["dataset"]["sessions"]:
        session_file = os.path.join(data_dir, session, f"data_{split}.hdf5")
        if not os.path.exists(session_file):
            continue

        # Use helper from repo to extract HDF5 contents
        data = load_h5py_file(session_file, b2txt_csv_df)
        n_trials = len(data["neural_features"])
        print(f"  - {session}: {n_trials} trials")

        for i in tqdm(range(n_trials), desc=f"{session}", unit="trial"):
            seq_class_ids = data["seq_class_ids"][i]
            seq_len = data["seq_len"][i]
            sentence_label = data["sentence_label"][i]

            # skip if label missing
            if sentence_label is None or seq_class_ids is None:
                continue

            # decode phonemes
            phoneme_seq = decode_phoneme_sequence(seq_class_ids, seq_len)
            text_label = sentence_label.decode("utf-8") if isinstance(sentence_label, bytes) else str(sentence_label)
            text_label = text_label.strip().lower()

            records.append({
                "split": split,
                "session": data["session"][i],
                "block_num": data["block_num"][i],
                "trial_num": data["trial_num"][i],
                "phoneme_sequence": phoneme_seq,
                "sentence": text_label
            })

# Save to CSV
df = pd.DataFrame(records)
df.to_csv(output_file, index=False)
print(f"\nSaved {len(df)} phoneme–text pairs to: {output_file}")

print("\nSample:")
print(df.sample(5))



Processing split: train
  - t15.2023.08.11: 288 trials


t15.2023.08.11: 100%|██████████| 288/288 [00:00<00:00, 169800.33trial/s]


  - t15.2023.08.13: 348 trials


t15.2023.08.13: 100%|██████████| 348/348 [00:00<00:00, 137557.04trial/s]


  - t15.2023.08.18: 197 trials


...

  - t15.2025.03.30: 30 trials


t15.2025.03.30: 100%|██████████| 30/30 [00:00<00:00, 82836.81trial/s]


  - t15.2025.04.13: 25 trials


t15.2025.04.13: 100%|██████████| 25/25 [00:00<00:00, 101213.90trial/s]


Saved 9498 phoneme–text pairs to: data/t15_pretrained_rnn_baseline/phoneme_text_training_data_final.csv

Sample:
      split         session  block_num  trial_num  \
3562  train  t15.2023.10.22          2         12   
2344  train  t15.2023.09.29          9          3   
6697  train  t15.2024.04.28         10         28   
3165  train  t15.2023.10.13          4         18   
5406  train  t15.2024.03.03          3         49   

                                       phoneme_sequence  \
3562  G UH D  |  L AH K  |  AA N  |  Y AO R  |  G R ...   
2344  JH AH S T  |  AH B AW T  |  EH N IY  |  K AY N...   
6697             D UW  |  Y UW  |  HH AE V  |  IH T  |    
3165  AY  |  W AA Z  |  IH N  |  DH AH  |  S ER V AH...   
5406  AY  |  HH OW P  |  M AY  |  N ER S  |  IH Z  |...   

                             sentence  
3562    good luck on your graduation.  
2344    just about any kind of music.  
6697                  do you have it?  
3165            i was in the service.  
5406  i hope